# --- TO DELETE ---

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns

from pyspark.sql import SparkSession, DataFrame

In [2]:
data = ['2021-03-08','2021-04-02','2021-04-03','2021-04-04','2021-04-05','2021-04-25','2021-06-14','2021-09-24','2021-11-02','2021-12-25','2021-12-26','2021-12-27','2021-12-28',
       '2022-01-01','2022-01-03','2022-01-26','2022-03-14','2022-04-15','2022-04-16','2022-04-17','2022-04-18','2022-04-25',
       '2022-06-13','2022-09-22','2022-09-23']
dates = pd.date_range(start='2021-02-01', end='2022-10-31')

In [3]:
import datetime
import re
from pyspark.sql.functions import *
from pyspark.sql.types import DateType

In [4]:
sp = SparkSession.builder.appName("FE-extras").getOrCreate()
sp

22/10/04 23:57:05 WARN Utils: Your hostname, J-L resolves to a loopback address: 127.0.1.1; using 172.18.71.108 instead (on interface eth0)
22/10/04 23:57:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/04 23:57:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/04 23:57:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/04 23:57:08 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [5]:
list1 = []
for date in dates:
    if date in data:
        list1.append((date.strftime('%Y-%m-%d'), 1))
    else:
        list1.append((date.strftime('%Y-%m-%d'), 0))
holiday_data = sp.createDataFrame(pd.DataFrame(list1, columns =['date', 'holiday']))
holiday_data = holiday_data.withColumn("date",col("date").cast(DateType()))
# print(holiday_data.printSchema())

In [7]:
holiday_data.show(2)

+----------+-------+
|      date|holiday|
+----------+-------+
|2021-02-01|      0|
|2021-02-02|      0|
+----------+-------+
only showing top 2 rows



In [10]:
holiday_data.write.parquet("../data/tables/holiday")

### Sector stuff

In [15]:
Administrative_Support_Services= ["equipment tool furniture appliance rent al leasing", "florist supply nursery stock flower", "lawn garden supply outlet including nursery"]
Personal_Services= ["shoe shop", "gift card novelty souvenir shop", "antique shop sale repair restoration service", "watch clock jewellery repair shop", "jewellery watch clock silverware shop",  "motor vehicle supply new part", "furniture home furnishing equipment shop manufa...", "tent awning shop", "optician optical good eyeglass"]
Arts_Recreation_Services = ["digital good book movie music", "music shop musical instrument piano sheet music", "health beauty spa", "bicycle shop sale service", "art dealer gallery", "hobby toy game shop", "stationery office supply printing writing paper"]
Information_Media_Telecommunications = ["telecom", "computer programming data processing integrated...", "book periodical newspaper", "artist supply craft shop", "computer computer peripheral equipment software", "cable satellite pay television radio service"]

In [16]:
desc = []
tags = []
for ele in Administrative_Support_Services:
    desc.append("Administrative_Support_Services")
tags += Administrative_Support_Services
for ele in Personal_Services:
    desc.append("Personal_Services")
tags += Personal_Services
for ele in Arts_Recreation_Services:
    desc.append("Arts_Recreation_Services")
tags += Arts_Recreation_Services
for ele in Information_Media_Telecommunications:
    desc.append("Information_Media_Telecommunications")
tags += Information_Media_Telecommunications

to_df = {
    "sector" :  desc,
    "Description" : tags
}

In [ ]:
sectors = sp.createDataFrame(pd.DataFrame(to_df))
dataset = dataset.join(sectors, ['Description'])
dataset.createOrReplaceTempView('data')
dataset = dataset.join(spark.sql("""select sector, avg(description_avg) as sector_avg from data group by sector"""), ['sector'])
dataset.limit(5)

### Industry stuff

In [11]:
data = [['Administrative_Support_Services','21.3','15.9','58.9'],['Personal_Services','16.9','12.8','65.7'],['Arts_Recreation_Services','17.3','13.1','63.8'],['Information_Media_Telecommunications','18.6','14.8','59.8']]
industry_trends=pd.DataFrame(data, columns=['sector','entry_rate','exit_rate','survival_rate'])
industry_trends = sp.createDataFrame(industry_trends)
industry_trends.limit(5)

DataFrame[sector: string, entry_rate: string, exit_rate: string, survival_rate: string]

In [12]:
industry_trends.show()

+--------------------+----------+---------+-------------+
|              sector|entry_rate|exit_rate|survival_rate|
+--------------------+----------+---------+-------------+
|Administrative_Su...|      21.3|     15.9|         58.9|
|   Personal_Services|      16.9|     12.8|         65.7|
|Arts_Recreation_S...|      17.3|     13.1|         63.8|
|Information_Media...|      18.6|     14.8|         59.8|
+--------------------+----------+---------+-------------+



In [13]:
industry_trends.write.parquet("../data/tables/sector_data")

In [14]:
sectors = sp.read.option("inferSchema", True).parquet("../data/tables/sector_data/")
sectors.show()

+--------------------+----------+---------+-------------+
|              sector|entry_rate|exit_rate|survival_rate|
+--------------------+----------+---------+-------------+
|Information_Media...|      18.6|     14.8|         59.8|
|Administrative_Su...|      21.3|     15.9|         58.9|
|Arts_Recreation_S...|      17.3|     13.1|         63.8|
|   Personal_Services|      16.9|     12.8|         65.7|
+--------------------+----------+---------+-------------+

